# introduction
进阶CNN包括两种网络：NiN、GoogLeNet。与经典CNN不同的是，这两种网络侧重于将通道数作为特征数。卷积层后不再接多个全连接层，而是做一个全局求平均的池化操作，所谓全局池化，就是池化窗口大小等于图片大小，经过池化后，每个通道只剩一个值。k个通道就相当于k个特征，然后再接一个全连接层即可。<br>
- **NiN**  卷积层后紧跟 1 * 1 的卷积层，我们可以把1 * 1 的卷积层看作全连接层，其中空间维度【高和宽】上的每个元素相当于样本，通道相当于特征。1 * 1的卷积层就是从通道维度出发的全连接层。<br>
- **GoogLeNet**  在NiN网络基础上改进，包含并行连接单元的网络。<br>
